In [11]:

# pip install xformers
# pip install huggingface_hub
# pip install auto-gptq
# pip install optimum
# pip install bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-xmpmhnax
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-xmpmhnax
  fatal: unable to access 'https://github.com/huggingface/transformers.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-xmpmhnax did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-xmpmhnax did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess,

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             torchscript=True,
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model.cuda()

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacty of 5.77 GiB of which 29.31 MiB is free. Including non-PyTorch memory, this process has 4.67 GiB memory in use. Of the allocated memory 4.43 GiB is allocated by PyTorch, and 128.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [2]:
RAG_context = ""
User_query = ""
prompt = f"Given a dictionary with 'context' and 'question', answer the 'question' based on 'context' ---- [Context: '{RAG_context}'; Question: '{User_query}']"
prompt_template=f'''<s>[INST] {prompt} [/INST]
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])



*** Generate:


/home/hardk/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s><s>[INST] Given a dictionary with 'context' and 'question', answer the 'question' based on 'context' ---- [Context: 'a girl name sara loves to eat chocolate'; Question: Who loves to eat broccoli?] [/INST]
Answer: Sara does not love to eat broccoli.</s>
*** Pipeline:
<s>[INST] Given a dictionary with 'context' and 'question', answer the 'question' based on 'context' ---- [Context: 'a girl name sara loves to eat chocolate'; Question: Who loves to eat broccoli?] [/INST]
Answer: Sara does not love to eat broccoli.
